In [ ]:
# | default_exp classes.DomoCard

In [ ]:
# | exporti
from dataclasses import dataclass, field

import asyncio
import httpx
from nbdev.showdoc import patch_to

import domolibrary.utils.DictDot as util_dd
import domolibrary.utils.chunk_execution as ce

import domolibrary.routes.card as card_routes

import domolibrary.client.DomoAuth as dmda

In [ ]:
# | export
@dataclass
class DomoCard:
    id: str
    auth: dmda.DomoAuth = field(repr=False)
    title: str = None
    description: str = None
    type: str = None
    urn: str = None
    chart_type: str = None
    dataset_id: str = None

    certification: field(default_factory=dict) = None
    owners: field(default_factory=list) = None

    def __post_init__(self):
        # self.Definition = CardDefinition(self)
        pass

    def display_url(self) -> str:
        return f"https://{self.auth.domo_instance}.domo.com/kpis/details/{self.id}"

    @classmethod
    async def _from_json(cls, card_obj, auth: dmda.DomoAuth):
        import domolibrary.classes.DomoUser as dmu
        import domolibrary.classes.DomoGroup as dmg

        dd = card_obj
        if isinstance(card_obj, dict):
            dd = util_dd.DictDot(card_obj)

        card = cls(
            auth=auth,
            id=dd.id,
            title=dd.title,
            description=dd.description,
            type=dd.type,
            urn=dd.urn,
            certification=dd.certification,
            chart_type=dd.metadata and dd.metadata.chartType,
            dataset_id=dd.datasources[0].dataSourceId if dd.datasources else None,
        )

        tasks = []
        for user in dd.owners:
            if user.type == "USER":
                tasks.append(dmu.DomoUser.get_by_id(user_id=user.id, auth=auth))
            if user.type == "GROUP":
                tasks.append(dmg.DomoGroup.get_by_id(group_id=user.id, auth=auth))

        card.owners = await ce.gather_with_concurrency(n=60, *tasks)

        return card

In [ ]:
# | exporti
@patch_to(DomoCard, cls_method=True)
async def get_by_id(
    cls: DomoCard,
    card_id: str,
    auth: dmda.DomoAuth,
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    res = await card_routes.get_card_metadata(
        auth=auth, card_id=card_id, debug_api=debug_api, session=session
    )

    if not res.is_success:
        raise Exception("unable to retrieve card {card_id}")

    domo_card = await cls._from_json(res.response, auth)

    return domo_card

In [ ]:
import os

token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

await DomoCard.get_by_id(card_id=1766265020, auth=token_auth)

DomoCard(id=1766265020, title='Vendor Amounts Dates', description='', type='kpi', urn='1766265020', chart_type='badge_vert_stackedbar', dataset_id='fd70d83b-38dc-40a2-9349-357ec4321d3e', certification=DictDot(state='NOT_CERTIFIED', adminCertified=False), owners=[DomoUser(id='1893952720', title=None, department='Business Improvement', display_name='Jae Wilson1', email_address='jae@onyxreporting.com', role_id=810756122, phone_number=None, web_landing_page=None, web_mobile_landing_page=None, employee_id=None, employee_number=None, hire_date=None, reports_to=None, publisher_domain=None, subscriber_domain=None, virtual_user_id=None)])

In [ ]:
# | exporti
@patch_to(DomoCard)
async def share(
    self: DomoCard,
    auth: dmda.DomoAuth = None,
    domo_users: list = None,  # DomoUsers to share card with,
    domo_groups: list = None,  # DomoGroups to share card with
    message: str = None,  # message for automated email
    debug_api: bool = False,
    session: httpx.AsyncClient = None,
):
    import domolibrary.routes.datacenter as datacenter_routes

    if domo_groups:
        domo_groups = domo_groups if isinstance(domo_groups, list) else [domo_groups]
    if domo_users:
        domo_users = domo_users if isinstance(domo_users, list) else [domo_users]

    res = await datacenter_routes.share_resource(
        auth=auth or self.auth,
        resource_ids=[self.id],
        resource_type=datacenter_routes.ShareResource_Enum.CARD,
        group_ids=[group.id for group in domo_groups] if domo_groups else None,
        user_ids=[user.id for user in domo_users] if domo_users else None,
        message=message,
        debug_api=debug_api,
        session=session,
    )

    return res

#### sample share_card

In [ ]:
import os
import domolibrary.client.DomoAuth as dmda
import domolibrary.classes.DomoGroup as dmg


token_auth = dmda.DomoTokenAuth(
    domo_instance="domo-community",
    domo_access_token=os.environ["DOMO_DOJO_ACCESS_TOKEN"],
)

domo_group = await dmg.DomoGroup.get_by_id(group_id=1324037627, auth=token_auth)
domo_group

domo_card = DomoCard(id=1766265020, auth=token_auth)

await domo_card.share(
    auth=token_auth,
    domo_groups=[domo_group],
    message=None,
    debug_api=False,
)

ResponseGetData(status=200, response='badge 1766265020 successfully shared with 1324037627', is_success=True)

In [ ]:
# | hide
import nbdev

nbdev.nbdev_export()

!nbqa black 50_DomoCard.ipynb